In [ ]:
import os
import json
import random

import pandas as pd
from lightning_whisper_mlx import LightningWhisperMLX
from tqdm import tqdm

from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
def write_json(data, filename):
    with open(filename, "w") as f:
        json.dump(data, f, ensure_ascii=False)

### 음성 데이터 변환

In [ ]:
model = "large-v3"

path_voice = "./data/voices"
list_voice = tqdm(os.listdir(path_voice))

whisper = LightningWhisperMLX(model=model, batch_size=12, quant=None)
result = []

for voice_file in list_voice:
    list_voice.set_description(voice_file)

    path_voice_file = os.path.join(path_voice, voice_file)
    translate = whisper.transcribe(audio_path=path_voice_file)

    resp = {"file_name": voice_file, "original": translate}

    write_json(resp, f"./data/output/{voice_file}.json")
    result.append(resp)

In [ ]:
import duckdb

df = duckdb.query("SELECT * FROM './data/output/*.json'").df()

df_file_name = df["file_name"]
df = pd.json_normalize(df["original"])
df = pd.concat([df_file_name, df], axis=1)
df

In [ ]:
df.loc[~df["language"].str.contains("ko")]

In [ ]:
# del df["segments"]

df.to_csv("./data/output.csv", index=False)

### 프롬프트 테스트

In [ ]:
prompt_system = """
<요청사항>
- 당신은 개인정보 보호를 위해 수정 요청을 받았습니다.
- 답변은 한국어(korean)로 작성 부탁드립니다.
- <검토 내용>을 확인하고 개인정보라고 판단되는 내용을 마스킹합니다. 
- 마스킹은 별표(*)로만 처리하고 개인정보를 지우셔야 됩니다. (예시: 홍길동 -> ***)
- 마스킹 후 <검토 내용> 전문 전체를 마스킹이 된 답변으로 제공해야 합니다.
- 미스킹을 하지않고 답변을 낼 경우 개인정보 보호법에 위반될 수 있습니다.
- 검토 내용을 요약하지 마세요

<마스킹해야 하는 개인정보 목록>
- 이름, 주민등록번호, 연락처, 주소, 이메일주소, 계좌번호, 신용카드번호, 생년월일, 성별, 소속, 사업자등록번호
"""

prompt_user = """
이하 내용 검토 후 한국어로 작성 부탁드립니다.

<검토 내용>
{input}
"""

In [ ]:
llm = ChatOllama(model="solar")
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", prompt_system),
        ("human", prompt_user),
    ]
)
chain = prompt | llm | StrOutputParser()

In [ ]:
df = pd.read_csv("./data/voc-result-whisper-large-v3.csv")

In [ ]:
text = df.iloc[random.randint(0, len(df))]["text"]
print(chain.invoke({"input": text}))